In [ ]:
#Note the significantly lower accuracy for FebHD which makes high accuracy unfeasible in Step 1
#I use fashion MNIST in this example because it's the easiest to import and be useable for
#the reader.

In [1]:
from sklearn import datasets
from keras.datasets import fashion_mnist
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from Module import *
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [3]:
def eigenvector_clusteriser(vectors:pd.Series, g):
    print(nx.info(g))
    comms = algorithms.eigenvector(g)
    comms = comms.communities
    comms = dict(sorted({item:list_id for list_id, l in enumerate(comms) for item in l}.items()))
    return pd.Series(comms)

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train=torch.from_numpy(x_train).reshape((-1,784)).float()
x_test=torch.from_numpy(x_test).reshape((-1,784)).float()
y_train=torch.from_numpy(y_train).long()
y_test=torch.from_numpy(y_test).long()
scaler = preprocessing.Normalizer().fit(x_train)
x_train = torch.tensor(scaler.transform(x_train))
x_test= torch.tensor(scaler.transform(x_test))

In [4]:
pass_1_classes=1000

In [119]:
# iris = datasets.load_iris()
# x_train=torch.tensor(iris.data).float()
# y_train=torch.tensor(iris.target).long()
# scaler = preprocessing.Normalizer().fit(x_train)
# x_train = torch.tensor(scaler.transform(x_train))

In [5]:
def get_classes(size_thresh,entries,x_train,y_train,dim):
    model1=HDCModel(500,len(x_train[0]),dim,classh=True)
    classifier=FebHDVersion2(64,.0001)
    classifier=TrainRepeater(classifier,5)
    print(x_train.dtype)
    train_start(model1,classifier,x_train,y_train)
    yhat1=model1(x_train)
    print(yhat1.unique())
    print("done with initial training")
    accs=torch.zeros(len(yhat1.unique()))
    sizes=torch.zeros(len(yhat1.unique()))
    classification=torch.zeros(len(yhat1.unique()))
    for i,ekans in enumerate(yhat1.unique()):
        # arbok=yhat2[2]y_train==yhat
        # print(ekans)
        # print(arbok)
        # print(y_train[yhat1==ekans].mode().values)
        # print(y_train[yhat1==ekans][:10])
        # print((y_train[yhat1==ekans]==y_train[yhat1==ekans].mode().values).float().mean())
        classification[i]=y_train[yhat1==ekans].mode().values
        istrue=(y_train[yhat1==ekans]==y_train[yhat1==ekans].mode().values).float()
        accs[i]=istrue.mean()
        sizes[i]=len(istrue)
    print("done with computing metrics")
    hypervectors=torch.zeros(len(yhat1.unique()),model1.dimensionality)
    for i,ekans in enumerate(yhat1.unique()):
        hypervectors[i]=model1.encode(x_train[yhat1==ekans]).sum(0)
    print("done with computing centroids")
    avgsims=torch.zeros(len(yhat1.unique()))
    for j,i in enumerate(yhat1.unique()):
        # ekans=yhat1[i]
        # arbok=yhat2[2]
        # print(ekans)
        # print(arbok)
        avgsims[j]=cos_cdist(model1.encode(x_train[yhat1==i]),hypervectors[j].unsqueeze(0)).mean()#ekans
    print("done with computing cosine deviation")
    is_optimized=sizes>size_thresh
    # entries=len(avgsims[is_optimized])
    order=-avgsims[is_optimized].argsort()
    print(sum(accs[is_optimized][order]*sizes[is_optimized][order])/sum(sizes[is_optimized][order]))
    yhat1.unique()[is_optimized][order][entries:]
    grouping=[]
    for i in range(0,entries):
        grouping.append((yhat1==yhat1.unique()[is_optimized][order][i]).nonzero().squeeze().tolist())
    print("done with computing groupings")
    print(grouping)
    return grouping

In [136]:
def uniformity_report(index_list):
    index_list=list(set(index_list))
    realvals=torch.zeros(len(index_list))
    for i, index in enumerate(index_list):
        realvals[i]=y_train[index]
    mode=realvals.mode()
    return mode.values, len(y_train[index_list][y_train[index_list]==mode.values])/len(index_list),len(index_list)

In [6]:
def combine_groupings(comparison_grouping, category):
    new_indices=[]
    for i in range(0,len(list(category))):
        new_indices+=comparison_grouping[list(category)[i]]
    return new_indices

In [7]:
import matplotlib.pyplot as plt
import networkx as nx

def get_graph_from_adjacency_matrix(adjacency_matrix):
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    return gr

In [9]:
#Step 1
#Note the significantly lower accuracy for FebHD which makes high accuracy unfeasible
dim=2000

size_thresh=25
entries=10

# sum(accs[is_optimized]*sizes[is_optimized])/sum(sizes[is_optimized])
# original_grouping=get_classes(size_thresh,entries,x_train,y_train,dim)

comparison_grouping=[]
for i in range(0,2):
    comparison_grouping+=get_classes(size_thresh,entries,x_train,y_train,dim)


similarity_matrix=torch.zeros(len(comparison_grouping),len(comparison_grouping))
for i,list_ in enumerate(comparison_grouping):
    for j,comparison_list in enumerate(comparison_grouping):
        similarity_matrix[i][j]=len(set(list_).intersection(comparison_list))

adjacency_matrix=similarity_matrix>=1

gr=get_graph_from_adjacency_matrix(adjacency_matrix)
connected_components=nx.connected_components(gr)
subgraphs=[gr.subgraph(c) for c in connected_components]


torch.float32
tensor([  3,   4,   7,   9,  12,  13,  16,  17,  19,  21,  22,  24,  28,  29,
         30,  31,  35,  36,  37,  40,  46,  47,  48,  49,  50,  51,  54,  56,
         58,  59,  63,  64,  65,  67,  69,  70,  72,  73,  74,  77,  78,  80,
         84,  85,  86,  87,  88,  89,  91,  92,  95,  96,  97,  99, 100, 102,
        104, 107, 108, 110, 115, 116, 117, 119, 120, 121, 122, 123, 125, 126,
        128, 129, 133, 134, 136, 140, 142, 143, 144, 145, 146, 147, 148, 149,
        150, 153, 158, 162, 163, 164, 165, 167, 168, 169, 172, 174, 175, 176,
        177, 178, 179, 181, 182, 183, 184, 185, 190, 191, 192, 195, 196, 197,
        199, 202, 203, 205, 206, 207, 208, 209, 211, 212, 213, 214, 215, 218,
        219, 220, 222, 224, 226, 228, 230, 231, 235, 238, 239, 240, 241, 242,
        243, 247, 250, 252, 253, 255, 256, 259, 262, 264, 266, 268, 269, 270,
        273, 274, 275, 279, 282, 283, 284, 286, 288, 289, 291, 292, 293, 294,
        297, 298, 299, 301, 302, 303, 305, 308, 31

In [11]:
categories=nx.community.louvain_communities(gr, threshold=1e-0, seed=None)
print(len(categories))
sum_=0
denom=0
clusters=[]
for i in range(0,len(categories)):
    print(len(comparison_grouping))
    print(comparison_grouping)
    print(categories)
    cluster=combine_groupings(comparison_grouping,categories[i])
    denom+=len(cluster)*uniformity_report(cluster)[1]
    sum_+=len(cluster)
    print(uniformity_report(cluster))
    print(len(cluster))
    clusters.append(cluster)
print(denom/sum_)

15
20
[[61, 237, 302, 354, 523, 549, 564, 595, 732, 825, 1013, 1101, 1158, 1216, 1599, 1849, 1929, 1950, 1979, 2041, 2202, 2286, 2300, 2560, 2695, 2707, 2712, 2716, 2730, 2896, 3041, 3050, 3237, 3480, 3524, 3548, 3554, 3560, 3585, 3654, 3674, 3717, 3743, 3938, 4031, 4234, 4325, 4499, 4546, 4585, 4784, 4796, 4817, 4821, 4879, 5094, 5118, 5123, 5358, 5433, 5511, 5733, 5774, 5790, 5802, 5908, 6088, 6215, 6244, 6251, 6290, 6309, 6310, 6472, 6488, 6502, 6732, 6747, 6956, 7071, 7125, 7162, 7199, 7276, 7400, 7419, 7423, 7512, 7583, 7770, 7980, 8105, 8127, 8259, 8273, 8389, 8504, 8522, 8601, 8648, 8812, 8850, 8923, 9127, 9131, 9188, 9197, 9205, 9249, 9334, 9710, 9745, 9865, 9954, 9978, 10280, 10396, 10430, 10568, 10768, 10782, 10801, 10873, 10914, 10926, 10981, 11031, 11231, 11260, 11463, 11488, 11517, 11572, 11626, 11696, 11718, 11779, 11797, 12116, 12162, 12237, 12277, 12344, 12444, 12478, 12482, 12494, 12502, 12517, 12679, 12754, 12778, 12799, 12816, 12943, 12954, 13121, 13155, 13417, 13671

NameError: name 'uniformity_report' is not defined

In [72]:
for i in range(len(clusters)):
    clusters[i]=torch.tensor(clusters[i])
category_membership=torch.zeros(len(x_train))
for index in range(len(x_train)):
    assignment_vector=torch.zeros(len(clusters))
    for i in range(len(clusters)):
        assignment_vector[i]=len(clusters[i][clusters[i]==index])

    category_membership[index]=assignment_vector.argmax()

In [90]:
model=HDCModel(classes=len(clusters),features=len(x_train[0]),dim=2000,classh=True)
classifier=ClassicVersion1(64,10)
# classifier=ClassicVersion0(64,1)
train_start(model,classifier,x_train,category_membership.long())

In [91]:
eval_acc(model,x_train,category_membership)

tensor(1.)

In [92]:
len(category_membership[category_membership==2])

50

In [93]:
yhat=model(x_train)

In [94]:
lengths=torch.zeros(len(model.classes))

for i in range(0,len(model.classes)):
    lengths[i]=len(yhat[yhat==i])
    

In [95]:
model.classes=model.classes[torch.topk(lengths,3).indices]

In [96]:
yhat=model(x_train)

In [97]:
eval_inferred_acc(model,x_train,y_train)

IndexError: mode(): Expected reduction dim 0 to have non-zero size.

In [68]:
while len(model.classes)!=10:
len(model.classes)

14

5633

In [14]:
category_membership.unique()

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 11., 12.])

In [52]:
len(category_membership[category_membership==12])

6562

In [18]:
from sklearn.metrics import fowlkes_mallows_score
fowlkes_mallows_score(category_membership,y_train)

0.8192413342522029

In [15]:
labels=category_membership
yhat=torch.zeros(y_train.shape)
modes=[]
for i in torch.unique(y_train):
    n=torch.mode(y_train[labels==i]).values
    modes.append(n)
for i in range(y_train.shape[0]):
    yhat[i]=modes[int(labels[i])]
eval=[yhat[i]==y_train[i] for i in range(len(y_train))]
print(sum(eval)/len(y_train))

IndexError: list index out of range

In [ ]:
for i in range(0,len(clusters)):
    print(len(labels[labels==i]))

6595
5152
5964
5897
6967
5950
5319
5894
6207
6055


In [ ]:
category_membership.shape

torch.Size([60000])

In [31]:
similarities=cos_cdist(model.encode(x_train), model.classes)

In [21]:
print(similarities.shape)

torch.Size([60000, 14])


In [36]:
similarities=similarities.numpy()

In [37]:
reduced_dim_vec = similarities
vectors_list = np.array_split(reduced_dim_vec, len(reduced_dim_vec))
category_assignments=pd.Series(vectors_list).apply(lambda x: x.squeeze())

In [38]:
category_assignments

0        [0.57468, 0.6285479, 0.5455284, 0.5325451, 0.5...
1        [0.5372889, 0.5457319, 0.650884, 0.539474, 0.5...
2        [0.5048593, 0.5196212, 0.4901391, 0.5671302, 0...
3        [0.5663998, 0.54452395, 0.51290345, 0.5775483,...
4        [0.58430296, 0.5444071, 0.5390455, 0.61799, 0....
                               ...                        
59995    [0.6090697, 0.56485844, 0.52651197, 0.5605037,...
59996    [0.6055812, 0.6474652, 0.53547484, 0.5355785, ...
59997    [0.55851996, 0.57203615, 0.54585, 0.54019326, ...
59998    [0.5564495, 0.5572184, 0.56590635, 0.5607771, ...
59999    [0.58963406, 0.5420234, 0.55819845, 0.57017237...
Length: 60000, dtype: object